# Análisis Exploratorio de Datos - Tesis de Maestría

Este notebook te permite explorar y analizar tus datos de forma interactiva.

**Autor**: Sistema de Análisis de Tesis  
**Fecha**: Diciembre 2025

## 1. Configuración e Importaciones

In [ ]:
# Importar librerías necesarias
import sys
from pathlib import Path

# Agregar rutas
sys.path.insert(0, str(Path.cwd() / 'src'))
sys.path.insert(0, str(Path.cwd() / 'config'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importar módulos del proyecto
from src.data_loader import DataLoader
from src.reliability_analysis import ReliabilityAnalyzer
from src.descriptive_stats import DescriptiveAnalyzer
from src.inferential_stats import InferentialAnalyzer
from src.visualization import DataVisualizer

# Configuración de visualización
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")

print("✓ Módulos importados correctamente")

## 2. Cargar Datos

In [ ]:
# Cargar datos desde SPSS
loader = DataLoader()

# Intenta cargar SPSS, si falla intenta Excel
try:
    data, metadata = loader.load_spss('BD/DatosAnalisisCOMPLETO.sav')
    print("✓ Datos cargados desde SPSS")
except:
    data = loader.load_excel('BD/Encuesta-LMS-CEV (respuestas) - Respuestas de formulario 1.xls')
    metadata = loader.metadata
    print("✓ Datos cargados desde Excel")

print(f"\nDimensiones: {data.shape[0]} filas × {data.shape[1]} columnas")

## 3. Vista Rápida de los Datos

In [ ]:
# Primeras filas
data.head()

In [ ]:
# Información general
data.info()

In [ ]:
# Resumen estadístico
data.describe()

## 4. Análisis de Fiabilidad

**IMPORTANTE**: Actualiza la lista `items` con tus variables reales.

In [ ]:
# Define tus ítems aquí (ejemplo)
items = ['P1', 'P2', 'P3', 'P4', 'P5']  # ← ACTUALIZA CON TUS VARIABLES

# Crear analizador
rel_analyzer = ReliabilityAnalyzer(data)

# Calcular Alpha de Cronbach
alpha_result = rel_analyzer.cronbach_alpha(items)

if alpha_result:
    print(f"\nAlpha de Cronbach: {alpha_result['alpha']:.4f}")
    print(f"Interpretación: {alpha_result['interpretation']}")
    print(f"\nNúmero de ítems: {alpha_result['n_items']}")
    print(f"Observaciones válidas: {alpha_result['n_observations']}")
    
    print("\nCorrelaciones ítem-total:")
    for item, corr in alpha_result['item_total_correlations'].items():
        print(f"  {item}: {corr:.3f}")

In [ ]:
# Visualizar Alpha si se elimina cada ítem
if alpha_result:
    alphas_del = pd.Series(alpha_result['alpha_if_item_deleted'])
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(alphas_del)), alphas_del.values, color='steelblue', alpha=0.7)
    plt.axhline(y=alpha_result['alpha'], color='red', linestyle='--', 
                label=f"Alpha actual: {alpha_result['alpha']:.3f}")
    plt.xticks(range(len(alphas_del)), alphas_del.index, rotation=45)
    plt.ylabel('Alpha de Cronbach')
    plt.xlabel('Ítem eliminado')
    plt.title('Alpha de Cronbach si se elimina cada ítem')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 5. Análisis Descriptivo

In [ ]:
# Crear analizador descriptivo
desc_analyzer = DescriptiveAnalyzer(data)

# Obtener variables numéricas
numeric_vars = data.select_dtypes(include=[np.number]).columns.tolist()
print(f"Variables numéricas encontradas: {len(numeric_vars)}")
print(numeric_vars[:10])  # Mostrar primeras 10

In [ ]:
# Estadísticas básicas de variables seleccionadas
# Cambia esta lista según tus necesidades
vars_to_analyze = numeric_vars[:5]  # Primeras 5 variables

stats = desc_analyzer.basic_statistics(vars_to_analyze)
stats

In [ ]:
# Matriz de correlación
corr_matrix = desc_analyzer.correlation_matrix(vars_to_analyze)

# Visualizar
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, fmt='.2f',
            vmin=-1, vmax=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Visualizaciones

In [ ]:
# Histogramas de las primeras variables
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, var in enumerate(vars_to_analyze[:6]):
    if i < len(axes):
        data[var].hist(bins=20, ax=axes[i], color='steelblue', 
                      alpha=0.7, edgecolor='black')
        axes[i].set_title(var, fontweight='bold')
        axes[i].set_xlabel('Valor')
        axes[i].set_ylabel('Frecuencia')
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Boxplot comparativo
plt.figure(figsize=(12, 6))
data[vars_to_analyze].boxplot()
plt.xticks(rotation=45, ha='right')
plt.ylabel('Valor')
plt.title('Diagrama de Cajas - Comparación de Variables', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Análisis Inferencial (Ejemplo)

Ajusta según tus hipótesis de investigación.

In [ ]:
# Crear analizador inferencial
inf_analyzer = InferentialAnalyzer(data, alpha=0.05)

# Ejemplo: Correlación entre dos variables
# ACTUALIZA con tus variables
if len(vars_to_analyze) >= 2:
    var1 = vars_to_analyze[0]
    var2 = vars_to_analyze[1]
    
    result = inf_analyzer.correlation_test(var1, var2, method='pearson')
    
    if result:
        print(f"\nCorrelación entre {var1} y {var2}:")
        print(f"Coeficiente: {result['correlation']:.4f}")
        print(f"Valor p: {result['p_value']:.6f}")
        print(f"Significativo: {'Sí' if result['significant'] else 'No'}")
        print(f"Fuerza: {result['strength']}")

## 8. Exportar Resultados

In [ ]:
# Exportar estadísticas a Excel
output_file = 'results/tablas/analisis_notebook.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    if stats is not None:
        stats.to_excel(writer, sheet_name='Estadísticas')
    
    if corr_matrix is not None:
        corr_matrix.to_excel(writer, sheet_name='Correlaciones')
    
    # Agregar más hojas según necesites

print(f"✓ Resultados exportados a: {output_file}")

## 9. Notas y Conclusiones

Escribe aquí tus observaciones e interpretaciones:

- 
- 
- 